# 5-1 Naive Bayes
# language : Python
https://www.youtube.com/watch?v=BqUmKsfSWho&list=PLqnslRFeH2Upcrywf-u2etjdxxkL8nl7E&index=5

## 0. 모델 기초

### Bayes Theorem
$$ P(A|B) = \frac{P(B|A) \times P(A)}{P(B)} $$


### In our case
$$ P(y|X) = \frac{P(X|y) \times P(y)}{P(X)} $$

### with feature vector X
$$ X = (x_1, x_2, x_3, \ldots ,x_n) $$

### Assume that all features are mutually independent
$$ P(y|X) = \frac{P(x_1|y) \cdot P(x_2|y) \cdots P(x_n|y) \cdot P(y)}{P(X)} $$

### Select class with highest probability
$$ \begin{align*} y &= arg \max_y P(y|X) \\  &= arg \max_y P(x_1|y) \cdot P(x_2|y) \cdots P(x_n|y) \cdot P(y) \\ &= arg \max_y log(P(x_1|y)) + log(P(x_2|y)) + \cdots + log(P(x_n|y)) + log(P(y))\end{align*} $$

## 1. 모델 구성 

In [29]:
import numpy as np

class NaiveBayes:

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # init mean, var, priors
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for c in self._classes:
            X_c = X[c==y]
            self._mean[c,:] = X_c.mean(axis=0)
            self._var[c,:] = X_c.var(axis=0)
            self._priors[c] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return y_pred

    def _predict(self, x):
        posteriors = []

        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            class_conditional = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

## 2. sample data 만들기 

In [32]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

X, y = datasets.make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

nb = NaiveBayes()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

print('Naive Bayes classification accuracy', accuracy(y_test, predictions))

Naive Bayes classification accuracy 0.965
